In [1]:
import msgpack
import open3d as o3d
# import trimesh
from IPython.display import display, Math
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
%config InlineBackend.figure_format = 'retina'

In [2]:
# https://adaickalavan.github.io/programming/udp-socket-programming-in-cpp-and-python/
# Python UDP Receiver

import socket
BUFFER_LEN = 200 #in bytes

def initUDP(IP, port):
    #Create a datagram socket
    sock = socket.socket(socket.AF_INET, # Internet
                         socket.SOCK_DGRAM) # UDP
    #Enable immediate reuse of IP address
    sock.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
    #Bind the socket to the port
    sock.bind((IP, port))
    #Set a timeout so the socket does not block indefinitely when trying to receive data
    sock.settimeout(1.5)

    return sock

def readUDP(sock):
    data, addr = sock.recvfrom(BUFFER_LEN)
    return data

In [3]:
ip_remote = "127.0.0.1"
port_local = 32000
port_remote = 32001
remote_address = (ip_remote,port_remote)

In [10]:
rec_sock = initUDP(ip_remote, port_local) # socket that receives udp packet
try:
    data_receive = readUDP(rec_sock)
#     print(data_receive)
    data_receive_unpacked = msgpack.unpackb(data_receive)
    print(data_receive_unpacked)
except socket.timeout:
    print("rec_socket timed out!")
rec_sock.shutdown(socket.SHUT_RDWR) # shutdown the connection
rec_sock.close()# close the connection

[11, 6.499999999999906, [0.2151199880877236, 0.9379999767747063, 0.798487112623547, -0.07515112046171543], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]


In [9]:
sen_sock = initUDP(ip_remote, port_local) # socket that send udp packet

data_send = [12,2,[0,0,1,2]]
data_send_packed = msgpack.packb(data_send)

sen_sock.sendto(data_send_packed,remote_address)

sen_sock.shutdown(socket.SHUT_RDWR) # shutdown the connection
sen_sock.close()# close the connection

In [94]:
data_send_packed

b'\x92\x01\x94\x00\x00\x01\x02'

In [86]:
from scipy.spatial.transform import Rotation as R
def RotateFromVec(vec, angle):
    vec = np.asarray(vec, dtype=np.float64)
    vec_norm = np.linalg.norm(vec)
    if vec_norm != 1:
        vec = vec/vec_norm
    vec *= angle
    h = np.eye(4)
    h[:3, :3] = R.from_rotvec(vec).as_dcm()
    return h       

# https://www.euclideanspace.com/maths/geometry/rotations/conversions/matrixToQuaternion/
def RotMatrixToQuarternion(m):
    """
    m is a 3x3 rotation matrix or 4x4 homogenious transformation matrix
    """
    w = np.sqrt(1+m[0,0]+m[1,1]+m[2,2])/2.0
    w4 = 4*w
    x = (m[2,1]-m[1,2])/w4
    y = (m[0,2]-m[2,0])/w4
    z = (m[1,0]-m[0,1])/w4
    return np.array([w,x,y,z])

# https://www.euclideanspace.com/maths/geometry/rotations/conversions/quaternionToAngle/index.htm
def QuarternionToAxisAngle(q):
    if(q[0]>1):
        q = q/np.linalg.norm(q)
    angle = 2.0 * np.arccos(q[0])
    s = np.sqrt(1-q[0]**2)
    if(s<1e-8):
        return q[1:],angle
    else:
        return q[1:]/s,angle

In [87]:
m = RotateFromVec([0,1,1],np.pi/2.)
q = RotMatrixToQuarternion(m)
QuarternionToAxisAngle(q)

(array([0.        , 0.70710678, 0.70710678]), 1.5707963267948966)